https://www.kaggle.com/code/cdeotte/tensorflow-transformer-0-790#TensorFlow-Transformer-Starter---LB-0.790

time series eda : https://www.kaggle.com/code/cdeotte/time-series-eda#Customer-Time-Series-EDA

Using a transformer requires 3D data(whereas Kaggle provideds 2d data as a csv.). The shape of the data is (#customers, 13, 188) which is (batchsize, sequence legnth, feature legnth). Each customer is a time series with 13 credit card statements. And each statemenr has 188 features. The data was created and saved to numpy files. 

Load Libraries

In [ ]:
import cupy, cudf
import numpy as np, pandas as pd
import matplotlib.pyplot as plt, gc, os

path_to_data = '../input/amax-data-for-transformers-and-rnns/data/'

train_model = True
path_to_model = './model/'

infer_Test = True

#Build Transformer Model

In [ ]:
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'  #tf will not use all memory

import tensorflow as tf
import tensorflow.keras.backend as k

from tensorflow import Keras
from tensorflow.kears import layers

print('using tensforflow version', tf.__version__)


In [ ]:
class TransformerBlock(layers.Layer) :
  def __init__(self, embed_dim, feat_dim, num_heads, ff_dim, rate=0.1) :

    super(TransformerBlock,self).__init__()
    self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
    self.ffn = keras.Sequential(
        [layers.Dense(ff_dim, activation='relu', ),
         layers.Dense(feat_dim),]
    )
    self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
    self.Dropout1 = layers.Dropout(rate)
    self.Dropout2 = layers,Dropout(rate)

  def call(self, inputs, training) :
    attn_output = self.att(inputs, inputs)
    attn_output = self.Dropout1(attn_output, training_training)

    out1 = self.layernorm1(inputs + attn_output)
    ffn_output = self.ffn(out1)
    ffn_output = self.dropout2(ffn_output, training_training)
    return self.layernorm2(out1 + ffn_output)

In [ ]:
feat_dim = 188
embed_dim =64
num_heads = 4
ff_dim = 128

dropout_rate = 0.3
num_blocks =2

def build_model() :

  #input embedding layer
  inp = layer.Input(shape=(13,188))
  embeddings = []

  for k in range(11) :
    emb = layers.Embedding(10,4)
    embeddings.append(emd(inp[:,:,:k]))

  x = layers.Concatenate()([inp[:,:,11:]] + embeddings)
  x = layers.Dense(feat_dim)(x)

  #Transformer blocks
  for k in range(num_blocks) :
    x_old = x
    transformer_block = TrnasformerBlock(embed_dim, feat_dim, num_heads, ff_dim, dropout_rate)
    x = transofrmer_block(x)
    x = 0.9*x + 0.1*x_old   #skip connection


  x = layers.Dense(64, activation='relu')(x[:,-1,:])
  x = layers.Dense(32, activation='relu')(x)
  
  outputs = layers.Dense(1, activation='sigmoid')(x)

  model = keras.Model(inputs = inp, outputs=outputs)
  opt = tf.keras.optimizers.Adam(learning_rate=0.01)
  loss = tf.keras.losses.BinaryCrossentropy()
  model.compile(loss=loss, optimizer=opt)
  # or
  model.compile(loss = Binary_crossentropy, optimizer='adam')
  
  return model

#Define Learning Schedule

In [ ]:
import math
lr_start = 1e-6
lr_max = 1e-3
lr_min = 1e-6
lr_rampup_epochs =0
lr_sustrain_epochs = 0
epochs = 10

def lrfn(epoch) :
  if epoch < lr_rampup_epochs :
    lr = (lr_max - lr_start) / lr_rampup_epochs * epoch + lr_start
  elif epoch <lr_rampup_epochs + lr_sustain_epochs :
    lr = lr_max
  else :
    decay_total_epochs = epochs - lr_rampup_epochs - lr_sustrain_epochs -1
    decay_total_index = epoch - lr_rampp_epochs - lr_sustrain_epochs

    phase=  math.pi * decat_epoch_index / decay_total+epochs
    cosine_decay = 0.5 * (1 + math.cos(phase))

    lr = (lr_max - lr_min) * cosine_decay + lr_min

  return lr

rng = [i for i in range(epochs)]
lr_y = [lrfn(x) for x in rng]

plt.figure(figsize=(10,4))
plt.plot(rng, lr_y, '-o')
plt.xlabel('epoch') ; plt.ylabel('lr')

print('learning rate schedule : {:.3g} to {:.3g} to {:.3g}'.format(lr_y[0], max(lr_y), ly_y[-1]))

lr = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=True)

#Define Competition Metric

In [ ]:
def amex_metric_mod(y_true, y_hat) :

  labels = np.transpose(np.array([y_true, y_hat]))
  labels = labels[labels[:,1].argsort()[::-1]]

  weights = np.where(labels[:,0] == 0, 20, 1)

  cut_vals = labels[np.cumsum(weights) <= int(0.04*np.sum(weights))]

  top_four = np.sum(cut_vals[:,0] / np.sum(labels[:,0]))

  gini = [0,0]

  for i in [1,0] :
    labels = labels
    weight_random = np.cumsum(weight / np.sum(weight))

    total_pos = np.sum(labels[:,0] * weight)
    cum_pos_found = np.cumsum(labels[:,0] * weight)
    lorentz = cum_pos_found / total_pos

    gini[i] = np.sum((lorentz - weight_random) * weight)

  return 0.5 * (gini[1]/gini[0] + top_four)

#Train Model

In [ ]:
if train_model :
  # save true and oof

  true = np.array([])
  oof = np.array([])

  verbose = 2

  for fold in range(5) :

    #in dices of train and valid folds
    valid_idx =  [2*fold+1, 2*fold+2]
    train_idx = [x for x in [i for i in range(1,11) if x not in valid_idx]]

    print('===='*25)

    # read train data from disk
    x_train = []
    y_train = []

    for k in train_idx :
      x_train.append(np.load(f'{path_to_data}data_{k}.npy'))
      y_trainappend(np.read_parquet(f'{path_to_data}target_{k}.pqt'))

      x_train = np.concatenate(x_train, axis=0)
      y_train = np.concat(y_train).target.values

    # read valid data from disk
    x_valid = []
    y_valid = []

    for k in train_idx :
      x_valid.append(np.load(f'{path_to_data}data_{k}.npy'))
      y_valid.append(np.read_parquet(f'{path_to_data}target_{k}.pqt'))

      x_valid = np.concatenate(x_valid, axis=0)
      y_valid = np.concat(y_valid).target.values


    # build and train model

    k.clear_session()

    model = build_model()

    h = model.fit(x_train, y_train, 
                  validation_data = (x_valid, y_valid),
                  batch_size = 512, epochs = epochs, verbose=verbose,
                  callbacks = [lr])
    
    if not os.path.exist(path_to_model) : os.makedirs(path_to_model)

    model.save_weights(f'{path_to_model}transformer_fold_{fold+1}.h5')

    # infer valid data

    p = model.predict(x_valid, batch_size=512, verbose=verbose).flatten()

    print()
    print('fold {fold+1} cv = '.format(amex_metric_mod(y_valid,p)))
    print()

    true = np.concatenate([true, y_valid])
    oof =  np.concatenate([oof,p])

    # clean memory
    del model, x_trian, y_train, y_valid, x_valid, p
    gc.collect()

  print('='*25)

  